In [70]:
import requests
from dotenv import load_dotenv
import os
import json
import pandas as pd
from datetime import datetime
from datetime import timezone
from dateutil import tz

# function to load secrets
def secretFunc():
	load_dotenv()
	global TRANSIT_URL
	global TRANSIT_API_KEY
	global STOPCODES
	global DIRECTIONS
	global OPERATORS
	global STOPNAMES
	global OPENWEATHER_API_KEY
	global LATITUDE
	global LONGITUDE

	TRANSIT_URL = 'http://api.511.org/transit/StopMonitoring/'
	TRANSIT_API_KEY = os.environ['TRANSIT_API_KEY']
	OPENWEATHER_API_KEY = os.environ['OPENWEATHER_API_KEY']

	# update STOPCODES, OPERATORS, DIRECTIONS, STOPNAMES to add add'l stops

	STOPCODES = [13915, 13914, 14509, 14510]
	OPERATORS = ['SF','SF', 'SF', 'SF']
	DIRECTIONS = ['Inbound', 'Outbound', 'Inbound', 'Outbound']
	STOPNAMES = ['Stanyan', 'Stanyan', 'Folsom', 'Folsom']
	LATITUDE = os.environ['LAT']
	LONGITUDE = os.environ['LONG']

	return()

# function to use 511org API to find next arrivals at each of the provided STOPCODES
def getNextTransit():
	secretFunc()
	arrivals = list()
	for i, stop in enumerate(STOPCODES):
		r = requests.get(TRANSIT_URL, 
						 params = {'agency': OPERATORS[i], 
								   'api_key': TRANSIT_API_KEY,
								   'stopcode': stop})
		
		content = json.loads(r.content)

		stopInfo = pd.DataFrame.from_records(content)
		stopInfo = stopInfo['ServiceDelivery']['StopMonitoringDelivery']['MonitoredStopVisit']

		for arrival in stopInfo:
			# Get arrivalTime from JSON content
			arrivalTime = arrival['MonitoredVehicleJourney']['MonitoredCall']['ExpectedArrivalTime']

			# Convert arrivalTime into 12h Pacific time
			utc_datetime = datetime.fromisoformat(arrivalTime[:-1])

			# Define the timezones
			utc_tz = tz.gettz('UTC')
			pacific_tz = tz.gettz('America/Los_Angeles')
			currentTime = datetime.now(timezone.utc)

			# Set the UTC timezone for the datetime object
			utc_datetime = utc_datetime.replace(tzinfo = utc_tz)

			# Convert the datetime object to Pacific time
			pacific_datetime = utc_datetime.astimezone(pacific_tz)

			# Format the datetime object as a 12-hour time string
			time_str = pacific_datetime.strftime('%I:%M %p')
			
			# Calculte timeToArrival
			timeToArrival = pacific_datetime - currentTime
			timeToArrival = divmod(timeToArrival.seconds, 60)
			timeToArrival = f"{str(timeToArrival[0]).rjust(2,'0')}:{str(timeToArrival[1]).rjust(2,'0')}"

			# Get destination from JSON content
			destination = arrival['MonitoredVehicleJourney']['MonitoredCall']['DestinationDisplay']

			# Append [code, direction, time]  to list
			arrivals.append([STOPNAMES[i], DIRECTIONS[i], destination, time_str, timeToArrival, stop])

	# Convert nested list to DataFrame
	arrivals = pd.DataFrame(arrivals, columns=['stopnames', 'direction', 'destination','arrivalTime', 'timeToArrival', 'stopcode'])
	return(arrivals)


	''' 
	todo:
		- connect to RPLCD to print information
		- implement refreshment interval
	'''

# main function
if __name__ == '__main__':
	transitArrivals = getNextTransit()
	print(transitArrivals)

   stopnames direction           destination arrivalTime timeToArrival  \
0    Stanyan   Inbound     Caltrain/Ballpark    11:06 PM         04:23   
1    Stanyan   Inbound     Caltrain/Ballpark    11:22 PM         20:45   
2    Stanyan   Inbound     Caltrain/Ballpark    11:29 PM         27:18   
3    Stanyan   Inbound   Townsend and 5th St    11:55 PM         54:06   
4    Stanyan   Inbound   Townsend and 5th St    12:15 AM         74:10   
5    Stanyan   Inbound  Fourth St + Townsend    12:34 AM         92:13   
6    Stanyan   Inbound  Fourth St + Townsend    01:04 AM        122:13   
7    Stanyan  Outbound           Ocean Beach    11:10 PM         08:45   
8    Stanyan  Outbound           Ocean Beach    11:30 PM         28:29   
9    Stanyan  Outbound           Ocean Beach    11:50 PM         48:29   
10   Stanyan  Outbound           Ocean Beach    12:34 AM         92:46   
11   Stanyan  Outbound           Ocean Beach    01:04 AM        122:46   
12   Stanyan  Outbound           Ocean

In [71]:
test = transitArrivals.loc[0, 'arrivalTime':'timeToArrival']
test.loc['arrivalTime']

'11:06 PM'

In [72]:
inbound1 = transitArrivals.loc[0, 'arrivalTime':'timeToArrival']
inbound2 = transitArrivals.loc[1, 'arrivalTime':'timeToArrival']

print(inbound1.loc['arrivalTime'], inbound1.loc['timeToArrival'] + '\n' + inbound2.loc['arrivalTime'], inbound2.loc['timeToArrival'])

11:06 PM 04:23
11:22 PM 20:45


In [73]:
output1 = f" City: {transitArrivals.loc[0, 'arrivalTime']}"
output2 = f"Next: {transitArrivals.loc[1, 'arrivalTime']}"

print(output1, '\n', output2)
len(output1)

 City: 11:06 PM 
 Next: 11:22 PM


15

In [74]:
print(len('Humidity: 89%'))
print(len('Wind: 13.8 MPH'))

13
14


In [75]:
transitArrivals[transitArrivals['destination'].str.contains('Ocean Beach')]

,stopnames,direction,destination,arrivalTime,timeToArrival,stopcode
7,Stanyan,Outbound,Ocean Beach,11:10 PM,08:45,13914
8,Stanyan,Outbound,Ocean Beach,11:30 PM,28:29,13914
9,Stanyan,Outbound,Ocean Beach,11:50 PM,48:29,13914
10,Stanyan,Outbound,Ocean Beach,12:34 AM,92:46,13914
11,Stanyan,Outbound,Ocean Beach,01:04 AM,122:46,13914
12,Stanyan,Outbound,Ocean Beach,02:04 AM,182:46,13914
19,Folsom,Outbound,Ocean Beach,11:11 PM,09:19,14510
20,Folsom,Outbound,Ocean Beach,11:31 PM,29:19,14510
21,Folsom,Outbound,Ocean Beach,11:51 PM,49:19,14510


In [76]:
WEATHER_URL = 'http://api.openweathermap.org/data/2.5/onecall' 
WEATHER_API_KEY = OPENWEATHER_API_KEY
UNITS = 'imperial'
secretFunc()

response = requests.get(WEATHER_URL, params = {'lat': LATITUDE,
			   											  'long': LONGITUDE,
														  'units': UNITS,
														  'appid': WEATHER_API_KEY})

In [77]:
response.json()

{'cod': 401,
 'message': 'Invalid API key. Please see https://openweathermap.org/faq#error401 for more info.'}